In [1]:
import ifcopenshell
f = ifcopenshell.open(r"X:\nor\oppdrag\Trondheim\520\31\5203127\BIM\Konstruksjon\IFC\NB3_RIBarmering.ifc")
rebars = f.by_type('IfcReinforcingBar')

pset_name = 'IFC Armering'
property_name = 'Bar Diameter'

def get_propery_value(rebar, pset_name, p_name):
    for pset in rebar.IsDefinedBy:
        if pset.RelatingPropertyDefinition.Name == pset_name:
            for property in pset.RelatingPropertyDefinition.HasProperties:
                if property.Name == p_name:
                    return property.NominalValue[0]

colours = (
    ("darkolivegreen", "#556b2f"),
    ("saddlebrown", "#8b4513"),
    ("olivedrab", "#6b8e23"),
    ("seagreen", "#2e8b57"),
    ("maroon", "#800000"),
    ("midnightblue", "#191970"),
    ("slategray", "#708090"),
    ("green", "#008000"),
    ("peru", "#cd853f"),
    ("steelblue", "#4682b4"),
    ("navy", "#000080"),
    ("yellowgreen", "#9acd32"),
    ("lightseagreen", "#20b2aa"),
    ("indianred", "#cd5c5c"),
    ("limegreen", "#32cd32"),
    ("goldenrod", "#daa520"),
    ("darkseagreen", "#8fbc8f"),
    ("purple", "#800080"),
    ("darkorchid", "#9932cc"),
    ("red", "#ff0000"),
    ("darkorange", "#ff8c00"),
    ("gold", "#ffd700"),
    ("yellow", "#ffff00"),
    ("mediumblue", "#0000cd"),
    ("burlywood", "#deb887"),
    ("lime", "#00ff00"),
    ("springgreen", "#00ff7f"),
    ("royalblue", "#4169e1"),
    ("darksalmon", "#e9967a"),
    ("crimson", "#dc143c"),
    ("aqua", "#00ffff"),
    ("deepskyblue", "#00bfff"),
    ("blue", "#0000ff"),
    ("purple3", "#a020f0"),
    ("greenyellow", "#adff2f"),
    ("tomato", "#ff6347"),
    ("fuchsia", "#ff00ff"),
    ("palevioletred", "#db7093"),
    ("khaki", "#f0e68c"),
    ("cornflower", "#6495ed"),
    ("plum", "#dda0dd"),
    ("skyblue", "#87ceeb"),
    ("deeppink", "#ff1493"),
    ("violet", "#ee82ee"),
    ("palegreen", "#98fb98"),
    ("aquamarine", "#7fffd4"),
    ("hotpink", "#ff69b4"),
    ("pink", "#ffc0cb")
)

#import itertools

#rebar_groups = []
#rebar_keys = []
#for k,g in itertools.groupby(rebars, lambda x: get_propery_value(x, pset_name, property_name)):
#    rebar_groups.append(list(g))
#    rebar_keys.append(k)
    
from collections import defaultdict
rebar_dict = defaultdict(list)
print("Grouping rebar by property value:")
for k,v in zip([get_propery_value(rebar, pset_name, property_name) for rebar in rebars], rebars):
    rebar_dict[k].append(v)

num_groups = len(rebar_dict)
print(num_groups)


import math
colours = colours*math.ceil(num_groups/len(colours))
colours = colours[:num_groups]

rgb_colours = list()
for hex in colours:
    h = hex[1].lstrip('#')
    rgb = tuple(int(h[i:i+2], 16)/255 for i in (0, 2, 4))
    rgb_colours.append(f.createIfcColourRgb(hex[0], rgb[0], rgb[1], rgb[2]))

i=0
print("Creating styled items for rebar color..")
for k, group in rebar_dict.items():
    colour = rgb_colours[i]
    print("{} stk {} med farge {}".format(len(group), k, colour.Name))
    surfaceStyleShading = f.createIfcSurfaceStyleShading()
    surfaceStyleShading.SurfaceColour = colour
    surfaceStyle = f.createIfcSurfaceStyle(colour.Name, "BOTH",(surfaceStyleShading,))
    presStyleAssign = f.createIfcPresentationStyleAssignment((surfaceStyle,))
    i=i+1
    for rebar in group:
        item = rebar.Representation.Representations[0].Items[0]
        f.createIfcStyledItem(item, (presStyleAssign,), colour.Name)
print("Saving file..")
f.write(r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\Rebar-diameter-modiefied.ifc")
print("Complete")